In [25]:
import pymongo
from pymongo import MongoClient
from bson import BSON, decode_all
import json
from bson.json_util import loads
from pprint import pprint

client = MongoClient('localhost', 27017)
db = client['restaurant']

In [26]:
with open('transactions.json') as g:
    if db['transactions']:
        db['transactions'].drop()
    file_data = json.load(g)
    db.transactions.insert_many(file_data)

In [27]:
for collections in db.list_collection_names():
    print(collections)

transactions


In [34]:
print(db.restaurants.find_one({}))

None


In [24]:
for x in db.restaurants.aggregate([{"$group": {"_id": "$borough", "nb_restaurant": { "$sum": 1 }}}]):
    print(x)

In [23]:
for x in db.restaurants.aggregate([{"$match": { "grades.0.grade": "C" } },{"$group": {"_id": "$borough", "nb_restaurant": { "$sum": 1 }}}]):
    pprint(x)

In [22]:
for x in db.restaurants.aggregate([ {"$unwind": "$grades" }, {"$group": {"_id": "$borough", "moyenne": { "$avg": "$grades.score" } }}, {"$sort": {"moyenne": -1}}]):
    print(x)

In [13]:
print(db.transactions.find_one({}))
for x in db.transactions.aggregate([ {"$unwind": "$Transaction" }, {"$group": {"_id": "Total", "somme": { "$sum": "$Transaction.price" } }}]):
    print(x)

{'_id': ObjectId('602164049386e3323b77fb0a'), 'Id': 100, 'Name': 'John', 'TransactionId': 'tran1', 'Transaction': [{'ItemId': 'a100', 'price': 200}, {'ItemId': 'a110', 'price': 210}], 'Subscriber': True, 'Payment': {'Type': 'Credit-Card', 'Total': 400, 'Success': True}, 'Note': '1st Complete Record'}
{'_id': 'Total', 'somme': 1640}


In [17]:
for x in db.transactions.aggregate([ {"$unwind": "$Transaction" }, {"$group": {"_id": "$TransactionId", "somme": { "$sum": "$Transaction.price" } }}]):
    print(x)

{'_id': 'tran2', 'somme': 410}
{'_id': 'tran1', 'somme': 410}
{'_id': 'tran3', 'somme': 410}
{'_id': 'tran4', 'somme': 410}


In [18]:
for x in db.transactions.aggregate([ {"$unwind": "$Transaction" }, {"$group": {"_id": "$Payment.Type", "somme": { "$sum": "$Transaction.price" } }}]):
    print(x)

{'_id': 'Debit-Card', 'somme': 410}
{'_id': 'Credit-Card', 'somme': 820}
{'_id': None, 'somme': 410}


In [19]:
print(db.transactions.find({}).sort("Id", -1)[0])

{'_id': ObjectId('602164049386e3323b77fb0d'), 'Id': 103, 'Name': 'Dylan', 'TransactionId': 'tran4', 'Transaction': [{'ItemId': 'a100', 'price': 200}, {'ItemId': 'a110', 'price': 210}], 'Subscriber': True, 'Payment': None, 'Note': 'Payment is Null'}


In [20]:
print(db.transactions.find({"Transaction.price"})[0])

TypeError: filter must be an instance of dict, bson.son.SON, or any other type that inherits from collections.Mapping